In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn


from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [4]:
## HyperParameters 
lr = 0.001
training_steps = 10000
bs = 128
display_step = 400

# Network Parameters 
n_in = 28  #### One row of pixels, 28*28 (next row will be used as input at next time step in rnn)
timesteps = 28 
n_hidden = 128
n_out = 10


# Graph input
X = tf.placeholder("float", shape=[None, timesteps, n_in])
Y = tf.placeholder("float", shape=[None, n_out])

In [5]:
# Define weights 

weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_out]))
}

biases = {
    'out': tf.Variable(tf.random_normal([n_out]))
}

In [6]:
def RNN(x, weights, biases):
    
    ## Prepare the data shape to match "rnn" function requirements
    ## Current Data input shape: (batch_size , timesteps, n_inputs)
    
    ## required shape: 'timsteps' tensors list of shape (batch_size, n_input)
    ## so that the next input will be of next row 
    
    x = tf.unstack(x, timesteps, 1)
    
    # define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    
    # get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    
    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']



In [7]:
logits = RNN(X, weights, biases)

preds = tf.nn.softmax(logits)

# define loss
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
train_op = optimizer.minimize(loss_op)


## Evaluate the model
correct_pred = tf.equal(tf.argmax(preds,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

## Initialize the variables
init = tf.global_variables_initializer()


Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [ ]:
# Training
with tf.Session() as sess:
    sess.run(init)
    
    for step in range(1, training_steps):
        batch_x, batch_y = mnist.train.next_batch(bs)
        
        batch_x = batch_x.reshape((bs, timesteps, n_in))
        
        sess.run(train_op, feed_dict={X:batch_x, Y:batch_y})
        
        if(step % display_step == 0):
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X:batch_x, Y:batch_y})

            print("step ", str(step), "Minibatch Loss: ", loss, "Accuracy: ", acc)
    
    print("Training Done")
    
    ## calcualte accuracy  for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, timesteps, n_in))
    test_label = mnist.test.labels[:test_len]
    acc = sess.run(accuracy, feed_dict={X:test_data, Y:test_label})
    print("test Accuracy",  acc)

step  400 Minibatch Loss:  2.028318 Accuracy:  0.3671875
step  800 Minibatch Loss:  1.8318708 Accuracy:  0.390625
step  1200 Minibatch Loss:  1.6105896 Accuracy:  0.4765625
step  1600 Minibatch Loss:  1.3530831 Accuracy:  0.5546875
step  2000 Minibatch Loss:  1.2823751 Accuracy:  0.578125
step  2400 Minibatch Loss:  1.1713055 Accuracy:  0.625
step  2800 Minibatch Loss:  1.2694952 Accuracy:  0.6640625
step  3200 Minibatch Loss:  0.96380067 Accuracy:  0.75
step  3600 Minibatch Loss:  1.0202482 Accuracy:  0.7109375
step  4000 Minibatch Loss:  0.8332682 Accuracy:  0.734375
step  4400 Minibatch Loss:  0.95905715 Accuracy:  0.6875
